# Cruncher
### Amazon Advertising Data Aggregator and Bid Adjustment catalyst
The goal of this tool is to merge various data sources into succinct and focused PPC Audit spreadsheets

In [31]:
# Dependencies

import pandas as pd
import numpy as np
#import sys
import os
import glob
import openpyxl

from openpyxl import Workbook,load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

from numpy.polynomial import Polynomial as Poly

from datetime import date

# Create Time Stamp
today = date.today()
td_form = today.strftime("%b-%d-%Y")

# Set option to regard empty string as NaN
pd.set_option('mode.use_inf_as_na', True)

import re

In [32]:
# Functions for File management

#Fn to read a tab of the bulksheet into a df
def readbs(path,tab):
    newpath = f'{path}*.xlsx'
    this = glob.glob(newpath)
    if (len(this)>1):
        print(f'Warn: Too many bulksheet file ({len(this)}).. Using first item..')
    return pd.read_excel(this[0], tab)

def readbs_sp(path,tab):
    newpath = f'{path}*.xlsx'
    this = glob.glob(newpath)
    if (len(this)>1):
        print(f'Warn: Too many bulksheet file ({len(this)}).. Using first item..')
    return pd.read_excel(this[0], tab, dtype={'Campaign Id':'string','Ad Group Id':'string',
                'Keyword Id (Read only)':'string','Product Targeting Id (Read only)':'string'})

#Fn to read the first sheet of a single excel doc with a folder reference
def readis(path,enc):
    newpath = f'{path}*.csv'
    this = glob.glob(newpath)
    if (len(this)>1):
        print(f'Warn: Too many csv file ({len(this)}).. Using first item..')
    return pd.read_csv(this[0], encoding=enc)

In [33]:
# Functions for Accepting User Input

#Fn for onboarding new ad group detected in bulksheet
def new_ag(aga):
    new_aga = None
    for idx, row in aga.iterrows():
        add = row
        add['Product Name'] = add['Ad Group Name'].split(' - ')[0]
        p = f'New Ad Group Detected: {os.linesep} {add["Ad Group Name"]} {os.linesep} Would you like to add it? .. (Y/nae)'
        r = input(p)
        if r == 'y' or r == 'Y' or r == 'yae' or r == 'Yae' or r == 'Yes' or r == 'yes':
            p = f'Enter Ad Group Type: {os.linesep} (i.e KW Broad, KW Phrase KW Exact, AUTO, CAT, ASIN Anc, ASIN Comp) {os.linesep} ..'
            add['Ad Group Type'] = input(p)
            p = f'Enter Product Name: {os.linesep} Expected:{add["Product Name"]}..'
            add['Product Name'] = input(p)
            if new_aga is None:
                new_aga = add
            else:
                new_aga = pd.concat([new_aga, add], axis=1)
    return pd.DataFrame(new_aga).transpose()

In [34]:
# Paths Setup
print('Updating Paths..')

ddir = 'Data/'
rdir = 'Resources/'
bsdir = f'{ddir}Bulksheet/'
h10dir = f'{ddir}H10_Keyword_History/'
spdir = f'{ddir}SP_Reports/'
sbdir = f'{ddir}SB_Reports/'
sbvdir = f'{ddir}SBV_Reports/'
confdir = f'{rdir}Config/'
hist_path = f'{rdir}History/adjustment_history.csv'

Updating Paths..


In [35]:
# Pull in Current Sponsored Products Reports
print('Updating Bulksheet for Sponsored Products..')
raw_sp = readbs_sp(bsdir,'Sponsored Products Campaigns')

print('Updating Targeting 90D for Sponsored Products..')
sp_targ = readbs(f'{spdir}90D_Targeting/',0)

print('Updating Searchterm IS for Sponsored Products..')
sp_is = readis(f'{spdir}30D_Searchterm_IS/','UTF-8')

Updating Bulksheet for Sponsored Products..


/home/kneel/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Updating Targeting 90D for Sponsored Products..


/home/kneel/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Updating Searchterm IS for Sponsored Products..


In [36]:
# Pull in Current Sponsored Brands Reports
print('Updating Bulksheet for Sponsored BRANDS..')
raw_sb = readbs_sp(bsdir,'Sponsored Brands Campaigns')

print('Updating Targeting 90D for Sponsored Brands..')
sb_targ = readbs(f'{sbdir}90D_Keyword/',0)

print('Updating Targeting 90D for Sponsored Brands Video..')
sbv_targ = readbs(f'{sbvdir}90D_Keyword/',0)

print('Updating Searchterm IS for Sponsored Brands..')
sb_is = readis(f'{sbdir}30D_Searchterm_IS/','UTF-8')

Updating Bulksheet for Sponsored BRANDS..


/home/kneel/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Updating Targeting 90D for Sponsored Brands..


/home/kneel/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Updating Targeting 90D for Sponsored Brands Video..


/home/kneel/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Updating Searchterm IS for Sponsored Brands..


In [37]:
# Import Config Csvs
prod_df = pd.read_csv(f'{confdir}products.csv', dtype={'Product Name':'string', 
       'Ad Type':'string', 'Campaign Id':'string', 'Ad Group Id':'string',
       'Campaign Name':'string', 'Ad Group Name':'string', 'Default Bid':'float64', 'Ad Group Type':'string',
       'ASIN':'string', 'SKU':'string', 'Total Cost':'float64', 'Pick Pack Fee':'float64', 'AMZ Price':'float64',
       'Referral Fee':'float64', 'BE ROAS':'float64', 'BE ACOS':'float64', 'Percentage':'float64', 'DP CVR':'float64'})
ag_df = pd.read_csv(f'{confdir}adgroups.csv', dtype={'Product Name':'string', 
       'Ad Type':'string', 'Campaign Id':'string', 'Ad Group Id':'string',
       'Campaign Name':'string', 'Ad Group Name':'string', 'Default Bid':'float64', 'Ad Group Type':'string',
       'ASIN':'string', 'SKU':'string', 'Total Cost':'float64', 'Pick Pack Fee':'float64', 'AMZ Price':'float64',
       'Referral Fee':'float64', 'BE ROAS':'float64', 'BE ACOS':'float64', 'Percentage':'float64', 'DP CVR':'float64'})


# Updates to Ad Group Config

# Check for new ad groups in bulk sheet for existing products
ag_add = raw_sp.loc[(raw_sp['Entity'] == 'Ad Group') & (raw_sp['Ad Group Id'].isin(ag_df['Ad Group Id']) == False),['Product','Campaign Id','Ad Group Id', 'Campaign Name (Informational only)', 'Ad Group Name (Informational only)', 'Ad Group Default Bid']].reset_index(drop=True)
ag_add.columns = ['Ad Type','Campaign Id','Ad Group Id','Campaign Name','Ad Group Name','Default Bid']
ag_add_sb = raw_sb.loc[(raw_sb['Entity'] == 'Campaign') & (raw_sb['Campaign Id'].isin(ag_df['Campaign Id']) == False) & (raw_sb['Campaign Name'].isin(ag_df['Campaign Name']) == False),['Product','Campaign Id','Ad Group Id (Read only)', 'Campaign Name (Informational only)', 'Campaign Name (Informational only)', 'Bid']].reset_index(drop=True)
ag_add_sb.columns = ['Ad Type','Campaign Id','Ad Group Id','Campaign Name','Ad Group Name','Default Bid']
ag_add = pd.concat([ag_add,ag_add_sb], axis=0)

row_no= ag_add.shape[0]
if (row_no>0):
    print(f'Warn: {row_no} new ad groups detected in campaigns.. Need to add to config to process..')

    screened = new_ag(ag_add)
    if screened is not None:
       ag_df = pd.concat([ag_df,screened])
else:
    print('No new ad groups..')

print('Updating Product Info..')
# Merge with Latest Product Info
#ag_df = pd.merge(ag_df,prod_df, on='Product Name', how='left').reset_index(drop=True)
ag_df = ag_df.set_index('Product Name')
ag_df.update(prod_df.set_index('Product Name'))
ag_df = ag_df.reset_index()

print('Updating Bid Multipliers..')
# First Get Bid Multipliers
mult = raw_sp[(raw_sp.Entity == 'Bidding Adjustment')].groupby(['Campaign Id']).Percentage.max()
# Merge multipliers to ad groups dataframe in memory
ag_df = ag_df.merge(mult, how='left', on='Campaign Id').reset_index(drop=True)
ag_df.Percentage_y = (ag_df.Percentage_y/100) + 1
ag_df['Percentage'] = ag_df.Percentage_y.fillna(ag_df.Percentage_x)

ag_df = ag_df.drop(['Percentage_x', 'Percentage_y'], axis=1)

ag_df.Percentage = ag_df.Percentage.fillna(1)

print('Writing Updates to AG config..')
ag_df.to_csv(f'{confdir}adgroups.csv', index=False)


No new ad groups..
Updating Product Info..
Updating Bid Multipliers..
Writing Updates to AG config..


In [38]:
#Converter for coercing datatype in h10 export
def rank_conv(v):
    try:
        return int(v)
    except ValueError:
        return np.nan

# Function for reading in H10 Rank csv files all at once
def rankfolder(path,enc,sub):
    dfa = None
    newpath = f'{path}*.csv'
    for f in glob.glob(newpath):
        if dfa is None:
            dfa = pd.read_csv(f, encoding=enc, converters={'Search Volume':rank_conv}, low_memory=False, parse_dates=['Date Added'])
            print('File read success..')
        else:
            dfb = pd.read_csv(f, encoding=enc, converters={'Search Volume':rank_conv}, low_memory=False, parse_dates=['Date Added'])
            dfa = pd.concat([dfa, dfb])
            #print('Concat Next File success..')
    print('finish concat.. cleaning dupes..')
    #print(dfa.head())
    return dfa.drop_duplicates(subset=sub).reset_index()

#Set subset for duplicate search
rank_subset = ['ASIN','Keyword','Date Added']

#Read in CSVS
rank_df = rankfolder(h10dir,'UTF-8',rank_subset)


# Function to parse groupby in preparation to merge
def rankagg(g):
    org = pd.Series(x for  x in g['Organic Rank'] if x !='>306').astype('string')
    spo = pd.Series(x for  x in g['Sponsored Position'] if x !='-').astype('string')
    length = spo.size
    if spo is not None and (length > 1):
        sp_slope = np.polyfit(np.arange(0,length,1),spo.astype('float'),deg=1)[0]
    else:
        sp_slope = np.nan
    return pd.Series({
        'Search Volume': g['Search Volume'].mean(),
        'Organic Rank': (', ').join(org),
        'Sponsored Rank': (', ').join(spo),
        'Organic Mean':org.astype('float').mean(),
        'Sponsored Mean':spo.astype('float').mean(),
        'Sponsored Slope':sp_slope
    })

# Group by unique Keyword/Product Combo and apply function to concat ranks
print('Combining Ranks to comma delimited timeseries')
rank_df = rank_df.sort_values('Date Added',ascending=False).groupby(['ASIN','Keyword'])['Search Volume','Organic Rank','Sponsored Position'].apply(rankagg)



File read success..
finish concat.. cleaning dupes..
Combining Ranks to comma delimited timeseries


/tmp/ipykernel_10649/3345205729.py:51: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rank_df = rank_df.sort_values('Date Added',ascending=False).groupby(['ASIN','Keyword'])['Search Volume','Organic Rank','Sponsored Position'].apply(rankagg)


In [39]:
# The Big Merge (SP)

# Merge BS and Ad Group info
merged = pd.merge(raw_sp[['Product', 'Entity', 'Campaign Id', 'Ad Group Id',
       'Keyword Id (Read only)', 'Product Targeting Id (Read only)',
       'State', 'Campaign State (Informational only)',
       'Ad Group State (Informational only)',  
       'Ad Group Default Bid (Informational only)',
       'Bid', 'Keyword Text', 'Match Type', 'Product Targeting Expression',
       'Impressions', 'Clicks', 'Click-through Rate', 'Spend', 'Sales',
       'Orders', 'Units', 'Conversion Rate', 'CPC', 'ROAS']],
       ag_df[['Ad Group Id', 'Product Name', 'Ad Type', 'Campaign Name',
       'Ad Group Name', 'Ad Group Type', 'ASIN', 'SKU','AMZ Price', 'BE ROAS',
       'BE ACOS', 'Percentage', 'Organic Target', 'Sponsored Target', 'DP CVR']], 
       on='Ad Group Id', how='left')

# Drop Rows without Targets
merged = merged[(merged.State == 'enabled') & (merged['Campaign State (Informational only)'] == 'enabled') & (merged['Ad Group State (Informational only)'] == 'enabled') & ((merged.Entity == 'Keyword') | (merged.Entity == 'Product Targeting'))]

#Resolve match type for products to match index of targeting report
merged['Match Type'] = merged['Match Type'].fillna('-')
# Add column to help match with st_is report
merged['st_resolve'] = merged.apply(lambda x: x['Keyword Text'] if x['Entity'] == 'Keyword' else re.compile(r'asin="([^"]*)"', re.IGNORECASE).match(x['Product Targeting Expression']).group(1).lower() if re.compile(r'asin="([^"]*)"', re.IGNORECASE).match(x['Product Targeting Expression']) else np.nan, axis=1) 
merged['bid_resolve'] = merged.Bid.fillna(merged['Ad Group Default Bid (Informational only)'])
# Drop Rows in Searchterm IS that don't help with bid audits
sp_is_clean = pd.merge(sp_is[['Customer Search Term',
       'Search Term Impression Rank', 'Search Term Impression Share','Campaign Name','Ad Group Name']],
       ag_df[['Campaign Name','Ad Group Name','Ad Group Type']], on=['Campaign Name','Ad Group Name'], how='left')
sp_is_clean = sp_is_clean[(sp_is_clean['Ad Group Type'] == 'KW Exact') | (sp_is_clean['Ad Group Type'] == 'ASIN Comp') | (sp_is_clean['Ad Group Type'] == 'ASIN Anc')].drop('Ad Group Type',axis=1)
sp_is_clean['Search Term Impression Share'] = sp_is_clean['Search Term Impression Share'].str.replace('%','').astype('float64')/100
sp_is_clean['Search Term Impression Rank'] = sp_is_clean['Search Term Impression Rank'].astype('int')

# Merge BS and Searchterm IS Report
merged = pd.merge(merged, sp_is_clean, left_on=['Campaign Name','Ad Group Name','st_resolve'], right_on=['Campaign Name','Ad Group Name','Customer Search Term'], how='left').drop('Customer Search Term', axis=1)

# Calculate ceiling bids using listing conversion rates and multipliers
# (if no DP CVR OR current bid is already over this number - we set a fallback of $6 just to avoid obscenely high system recs)
merged['bid_ceil'] = merged.apply(lambda x: ((x['DP CVR']*x['AMZ Price']*x['BE ACOS'])/x.Percentage) if (x['DP CVR'] is not None and (((x['DP CVR']*x['AMZ Price']*x['BE ACOS'])/x.Percentage) >= x.bid_resolve)) else (6/x.Percentage),axis=1)

# Drop Columns Unnecessary in Targeting Report
sp_targ_clean = sp_targ[['Campaign Name','Ad Group Name','Targeting','Match Type','Impressions','Cost Per Click (CPC)','Click-Thru Rate (CTR)','7 Day Conversion Rate','Total Return on Advertising Spend (ROAS)']]
sp_targ_clean.columns = ['Campaign Name','Ad Group Name','Targeting','Match Type','90D Impressions','90D CPC','90D CTR','90D CVR','90D ROAS']
sp_targ_clean['st_resolve'] = sp_targ_clean.apply(lambda x: re.compile(r'asin="([^"]*)"', re.IGNORECASE).match(x['Targeting']).group(1).lower() if re.compile(r'asin="([^"]*)"', re.IGNORECASE).match(x['Targeting']) else x['Targeting'], axis=1)
sp_targ_clean['Match Type'] = sp_targ_clean['Match Type'].str.lower()
s
sp_targ_clean.drop('Targeting', axis=1, inplace=True)

# Merge BS and Longterm Targeting KPI
merged = pd.merge(merged, sp_targ_clean, on=['Campaign Name','Ad Group Name','st_resolve','Match Type'], how='left')

# Add merge key to rank df to indicate only KW Exacts
rank_df['Ad Group Type'] = rank_df.apply(lambda x: 'KW Exact',axis=1)

#Merge BS and rank df
merged = pd.merge(merged,rank_df,left_on=['ASIN','Keyword Text','Ad Group Type'],right_on=['ASIN','Keyword','Ad Group Type'],how='left').drop_duplicates(subset=['Keyword Id (Read only)', 'Product Targeting Id (Read only)', 'st_resolve','Match Type','bid_resolve']).reset_index(drop=True)

/tmp/ipykernel_10649/2375991302.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_targ_clean['st_resolve'] = sp_targ_clean.apply(lambda x: re.compile(r'asin="([^"]*)"', re.IGNORECASE).match(x['Targeting']).group(1).lower() if re.compile(r'asin="([^"]*)"', re.IGNORECASE).match(x['Targeting']) else x['Targeting'], axis=1)
/tmp/ipykernel_10649/2375991302.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_targ_clean['Match Type'] = sp_targ_clean['Match Type'].str.lower()
/tmp/ipykernel_10649/23759913

False

In [40]:
# Big Merge 2 (SB and SBV)

# Merge BS and Ad Group info

merged_sb = pd.merge(raw_sb[['Product', 'Entity', 'Campaign Id','Campaign Serving Status (Informational only)',
       'Keyword Id (Read only)', 'Product Targeting Id (Read only)',
       'State', 'Bid', 'Keyword Text', 'Match Type', 'Product Targeting Expression',
       'Impressions', 'Clicks', 'Click-through Rate', 'Spend', 'Sales',
       'Orders', 'Units', 'Conversion Rate', 'CPC', 'ROAS']],
       ag_df[['Campaign Id', 'Product Name', 'Ad Type', 'Campaign Name',
       'Ad Group Name', 'Ad Group Type', 'ASIN', 'SKU','AMZ Price', 'BE ROAS',
       'BE ACOS', 'Percentage', 'DP CVR']], 
       on='Campaign Id', how='left')

# Drop Rows without Targets
merged_sb = merged_sb[(merged_sb.State == 'enabled')  & ((merged_sb['Campaign Serving Status (Informational only)'] == 'running') | (merged_sb['Campaign Serving Status (Informational only)'] == 'other')) & ((merged_sb.Entity == 'Keyword') | (merged_sb.Entity == 'Product Targeting'))]

# Add column to help match with st_is report
merged_sb['st_resolve'] = merged_sb.apply(lambda x: x['Keyword Text'] if x['Entity'] == 'Keyword' else re.compile(r'asin="([^"]*)"', re.IGNORECASE).match(x['Product Targeting Expression']).group(1).lower() if re.compile(r'asin="([^"]*)"', re.IGNORECASE).match(x['Product Targeting Expression']) else np.nan, axis=1) 
merged_sb['bid_resolve'] = merged_sb.Bid

# Drop Rows in Searchterm IS that don't help with bid audits
sb_is_clean = pd.merge(sb_is[['Customer Search Term',
       'Search Term Impression Rank', 'Search Term Impression Share','Campaign Name']],
       ag_df[['Campaign Name','Ad Group Type']], on=['Campaign Name'], how='left')
sb_is_clean = sb_is_clean[(sb_is_clean['Ad Group Type'] == 'KW Exact') | (sb_is_clean['Ad Group Type'] == 'ASIN Comp') | (sb_is_clean['Ad Group Type'] == 'ASIN Anc')].drop('Ad Group Type',axis=1)
sb_is_clean['Search Term Impression Share'] = sb_is_clean['Search Term Impression Share'].str.replace('%','').astype('float64')/100
sb_is_clean['Search Term Impression Rank'] = sb_is_clean['Search Term Impression Rank'].astype('int')

# Merge BS and Searchterm IS Report
merged_sb = pd.merge(merged_sb, sb_is_clean, left_on=['Campaign Name','st_resolve'], right_on=['Campaign Name','Customer Search Term'], how='left').drop('Customer Search Term', axis=1)


# Drop Columns Unnecessary in Targeting Report
sb_targ_clean = pd.concat([sb_targ[['Campaign Name','Targeting','Match Type','Impressions','Cost Per Click (CPC)','Click-Thru Rate (CTR)','14 Day Conversion Rate','Total Return on Advertising Spend (ROAS)']],sbv_targ[['Campaign Name','Targeting','Match Type','Impressions','Cost Per Click (CPC)','Click-Thru Rate (CTR)','14 Day Conversion Rate','Total Return on Advertising Spend (ROAS)']]])
sb_targ_clean.columns = ['Campaign Name','Targeting','Match Type','90D Impressions','90D CPC','90D CTR','90D CVR','90D ROAS']
sb_targ_clean['st_resolve'] = sb_targ_clean.Targeting
sb_targ_clean['Match Type'] = sb_targ_clean['Match Type'].str.lower()
sb_targ_clean.drop('Targeting', axis=1, inplace=True)

# Merge BS and Longterm Targeting KPI
merged_sb = pd.merge(merged_sb, sb_targ_clean, on=['Campaign Name','st_resolve','Match Type'], how='left')

merged_sb['Ad Group Name'] = merged_sb['Campaign Name']



In [41]:
# Combine SP and SB Data before applying Recos
merged = pd.concat([merged,merged_sb]).reset_index(drop=True)

In [42]:
# Apply Bid Recommendations based on KPI and add to dataframe

# Define Conditions for Flags and Recommendations
cons = [
    ((merged['90D CVR'].notna()) & (merged['90D CVR'] > 0) & (merged['ROAS'].notna()) & (merged['ROAS'] > 0.0) & (merged['BE ROAS'] > 0.0) & (merged['AMZ Price'] > 0.0) & (merged['ROAS'] <= merged['BE ROAS']) & (((merged['AMZ Price'] * merged['90D CVR']) / (merged['BE ROAS'] * merged['Percentage'])) <= merged['bid_resolve'])),
    ((merged['ROAS'] > 0.0) & (merged['BE ROAS'] > 0.0) & (merged['ROAS'] <= merged['BE ROAS']) & ((merged['CPC'] / merged['Percentage']) <= merged['bid_resolve'])),  
    ((merged['BE ROAS'] > 0.0) & (merged['ROAS'] >= merged['BE ROAS']) & (merged['Organic Rank'].notna()) & (merged['Sponsored Rank'].notna()) & (merged['Organic Mean'] > (merged['Organic Target']*1.05)) & (merged['Sponsored Mean'] > (merged['Sponsored Target']*1.05)) & (np.abs(merged['Sponsored Slope']) > 0.03) & (merged['Sponsored Slope'] < 0.0)),
    ((merged['BE ROAS'] > 0.0) & (merged['ROAS'] >= merged['BE ROAS']) & (merged['Search Term Impression Rank'].notna()) & ((merged['Search Term Impression Rank']) > 2)),
    ((merged['BE ROAS'] > 0.0) & (merged['ROAS'] >= merged['BE ROAS']) & (merged['CPC']>0.0) & (((merged['CPC']/merged['Percentage']) * 1.15) >= merged['bid_resolve'])),
    ((merged['BE ROAS'] > 0.0) & (merged['ROAS'] >= merged['BE ROAS'])),
    ((merged['90D Impressions'].notna()) & (merged['90D Impressions'] > 0.0) & (merged['90D CTR'] > 0.0) & (merged['90D CVR'].notna()) & (merged['90D CVR'] == 0) & (merged['Clicks'].notna()) & (merged['Clicks'] >= 8)),
    ((merged['90D Impressions'] > 0.0) & (merged['Units'] == 0) & (merged['Impressions'] < 100) & (merged['Clicks'] < 5)),
    ((merged['90D Impressions'] == 0.0) & (merged['Impressions'] == 0.0))
]

# Define FLag IDs for each Condition
flags = (
    '0901',
    '0900',
    '0401',
    '0402',
    '0402',
    '0400',
    '0700',
    '0101',
    '0100'
    )

# Define Flag Class for each Condition
types = (
    'Down',
    'Down',
    'Up',
    'Up',
    'Up',
    'Up',
    'Kill',
    'Stale',
    'Stale'
    )

# Define Flag Text for each Condition
texts = (
    'Unprofitable - Lower to Safe Bid',
    'Unprofitable - Missing Historical Data',
    'Profitable - Losing Sponsored Position',
    'Profitable - Low Impression Share',
    'Profitable - Competitive Bid', 
    'Profitable - Already Bidding High',
    'Many Clicks With No Sales',
    'Low Data - Raise Bid or Kill',
    'No Impressions - Raise Bid 5%'
    )

# Formulas to calculate reccomended Bids, conditionally
recs = (
    np.maximum(np.minimum(((merged['AMZ Price'] * merged['90D CVR']) / (merged['BE ROAS'] * merged['Percentage'])),(merged['bid_resolve']*0.95)),0.02),
    np.maximum(np.minimum((merged['CPC'] / merged['Percentage']),(merged['bid_resolve']*0.95)),0.02),
    merged['bid_resolve']*1.2,
    merged['bid_resolve']*1.2,
    merged['bid_resolve']*1.15,
    merged['bid_resolve']*1.10,
    0.02,
    merged['bid_resolve']*1.05,
    merged['bid_resolve']*1.05
)


In [43]:
# Generate Dataframe with proportional Shape to Merged DF
# We are using Numpy.select() in series for each items based on the same conditions.
# In testing this had significantly better performance than df.apply with lambda fns
reco_df = pd.DataFrame({'Flag':np.select(cons,flags),
            'Flag Type':np.select(cons,types),
            'Flag Text':np.select(cons,texts),
            'Recommended Bid':np.select(cons,recs)})

# Merge (column-wise) Merged DF and Recommendations
merged = pd.concat([merged,reco_df],axis=1)
merged['Recommended Bid'] = merged['Recommended Bid'].round(2)
merged['Max CPC'] = merged.bid_resolve * merged.Percentage


In [44]:
#Merge previous history to audits


# Read in History Csv
hist_df = pd.read_csv(hist_path, encoding='UTF-8', parse_dates=['Date'], dtype=merged.dtypes.to_dict()).sort_values('Date').drop_duplicates(subset=['Entity','Campaign Id','Ad Group Id','Keyword Id (Read only)',
                                                                                                                    'Product Targeting Id (Read only)','Keyword Text','Product Targeting Expression','Match Type'],
                                                                                                                    keep='last').reset_index()
hist_df['Change Date'] = hist_df['Date']
hist_df['Change Reason'] = hist_df['Flag Text']
hist_df['Change'] = hist_df.apply(lambda x: f'${x.Bid:,.2f} -> ${x["Will Set Bid to:"]:,.2f}', axis=1)

#Merge in Bid Edit History
merged = pd.merge(merged, hist_df[['Entity','Campaign Id',
                                'Keyword Id (Read only)','Product Targeting Id (Read only)',
                                'Keyword Text','Product Targeting Expression','Match Type','Change Date','Change Reason','Change','Notes']],
                                on=['Entity', 'Campaign Id', 'Keyword Id (Read only)', 
                                'Product Targeting Id (Read only)', 'Keyword Text', 'Product Targeting Expression','Match Type'],
                                how='left')

In [45]:
#Final Excel Audit Setup and Formating

# Drop Rows with no recommendations and reorder
audit = merged[merged.Flag != '0'][['Ad Group Name', 'Product Name', 'Ad Group Type', 'BE ROAS', 'Organic Target', 'Sponsored Target', 'DP CVR',
        'Product', 'Entity', 'Campaign Id', 'Ad Group Id', 'Keyword Id (Read only)', 'Product Targeting Id (Read only)', 'Keyword Text', 'Product Targeting Expression',
        'Campaign Name', 'st_resolve', 'Match Type', 'bid_resolve', 'Percentage', 'Max CPC', 
        'Impressions', 'Clicks', 'Orders', 'Units', 'Click-through Rate', 'Conversion Rate', 'Spend', 'Sales', 'CPC', 'ROAS', 
        '90D Impressions', '90D CTR', '90D CVR', '90D CPC', '90D ROAS',
        'Search Term Impression Rank', 'Search Term Impression Share', 'Organic Rank', 'Sponsored Rank', 'Organic Mean', 'Sponsored Mean',
        'Change Date','Change Reason','Change','Notes','Flag', 'Flag Type', 'Flag Text', 'Recommended Bid']].reset_index(drop=True)


#Reassign Column Names to show in spreadsheet
audit.columns = ['Ad Group','Product Name', 'AG Type', 'BE ROAS', 'Organic Target', 'Sponsored Target', 'DP CVR',
        'Product', 'Entity', 'Campaign Id', 'Ad Group Id', 'Keyword Id (Read only)', 'Product Targeting Id (Read only)', 'Keyword Text', 'Product Targeting Expression',
        'Campaign', 'Target', 'Match Type', 'Bid', 'Mult.', 'Max CPC', 
        'Impressions', 'Clicks', 'Orders', 'Units', 'CTR', 'CVR', 'Spend', 'Sales', 'CPC', 'ROAS', 
        '90D Impressions', '90D CTR', '90D CVR', '90D CPC', '90D ROAS',
        'IS Rank', 'IS %', 'Org. Rank', 'Spo. Rank', 'Org. Mean', 'Spo. Mean','Change Date','Change Reason','Change','Notes',
        'Flag', 'Flag Type', 'Flag Text', 'Reco. Bid']

# Add Actions Columns
audit['Skip?'] = False
audit['Override Bid'] = ''
audit['Will Set Bid to:'] = '=IF(NOT(ISBLANK(INDIRECT("RC[-1]",FALSE()))),IF(NOT(INDIRECT("RC[-2]",FALSE())),INDIRECT("RC[-1]",FALSE()),""),IF(NOT(INDIRECT("RC[-2]",FALSE())),INDIRECT("RC[-3]",FALSE()),""))'
audit['Multiplier'] = audit['Mult.']
audit['New CPC'] = '=IFERROR(ROUND(INDIRECT("RC[-1]",FALSE)*INDIRECT("RC[-2]",FALSE),2),"")'
audit ['AMZ Link']= '=IF(INDIRECT("RC[-47]",0)="Keyword",HYPERLINK("https://www.amazon.com/s?k="&SUBSTITUTE(INDIRECT("RC[-39]",0)," ","+")),IF(INDIRECT("RC[-47]",0)="Product Targeting",HYPERLINK("https://www.amazon.com/dp/"&INDIRECT("RC[-39]",0)),""))'
#set lists to iterate later
adtypes = ag_df['Ad Type'].unique()
agtypes = ag_df['Ad Group Type'].unique()
flagtypes = ['Up','Down','Kill','Stale']

#Short codes for ad types
short = {'Sponsored Products':'SP','Sponsored Brands': 'SB'}

# # Set Number Formats in Dataframe because it's less memory intensive than setting them in the spreadsheet
# format_mapping = {
#     'BE ROAS':'{:.2f}',
#     'Bid':"${:.2f}", 
#     'Mult.': "{:.0%}", 
#     'Max CPC':"${:.2f}", 
#     'Impressions': "{:,.0f}", 
#     'Clicks': "{:,.0f}", 
#     'Orders': "{:,.0f}", 
#     'Units': "{:,.0f}", 
#     'CTR': '{:.2%}', 
#     'CVR': '{:.2%}', 
#     'Spend':"${:,.2f}", 
#     'Sales':"${:,.2f}", 
#     'CPC':"${:.2f}", 
#     'ROAS':'{:.2f}', 
#     '90D Impressions': "{:,.0f}", 
#     '90D CTR': '{:.2%}', 
#     '90D CVR': '{:.2%}', 
#     '90D CPC':"${:,.2f}", 
#     '90D ROAS':'{:.2f}',
#     'IS %': '{:.2%}', 
#     'Org. Mean':'{:.2f}', 
#     'Spo. Mean':'{:.2f}',
#     'Reco. Bid':"${:.2f}",
#     'Multiplier': "{:.0%}"
# }
# for key, value in format_mapping.items():
#     audit[key] = audit[key].apply(value.format)

# Iterate thru products that exist
for product in prod_df['Product Name']:
    if audit['Product Name'].isin([product]).any().any():
        print(f'Starting Product: {product}')
        exist=None
        # Generate spreadsheet and product level details
        wb = load_workbook(filename=f'{confdir}sprint_template.xlsx')
        checklist = wb['Checklist']
        this_sh = wb['Temp']
        next_sh = wb.copy_worksheet(this_sh)
        
        # slice of audit for this product
        prod_aud = audit[audit['Product Name'] == product]
        ssname = f'Generated_Audits/{product}_{td_form}.xlsx'

        # Static Attributes for Product
        be = prod_df.loc[(prod_df['Product Name'] == product),'BE ROAS'].item()
        dpcr = prod_df.loc[(prod_df['Product Name'] == product),'DP CVR'].item()
        og = prod_df.loc[(prod_df['Product Name'] == product),'Organic Target'].item()
        sg = prod_df.loc[(prod_df['Product Name'] == product),'Sponsored Target'].item()
        
        # Iterate through Ad Type (SP, SB, etc.) and Ad group type (KW Exact, Competitor, Auto, etc.) and Flag type (Up Bid, Down Bid, Kill, etc.)
        for adtype in adtypes:
            for agtype in agtypes:
                for flagtype in flagtypes:
                    # Create slice for this combo
                    working = prod_aud.loc[(prod_aud['Product'] == adtype) & (prod_aud['AG Type'] == agtype) & (prod_aud['Flag Type'] == flagtype)]
                    working = working.sort_values(by='Spend',ascending=False).replace(np.nan, '', regex=True).astype('object')
                    l,w = working.shape
                    if (l>=1):
                        exist=True
                        #Set Top of Audit Static Details
                        this_sh['S1'] = product
                        this_sh['S2'] = adtype
                        this_sh['S3'] = agtype
                        this_sh['S4'] = flagtype
                        this_sh['Y1'] = be
                        this_sh['Y2'] = dpcr
                        this_sh['Y3'] = og
                        this_sh['Y4'] = sg
                        this_sh['Y5'] = l

                        #Add Audit to Checklist
                        checklist.append([f'{short[adtype]} - {agtype} - {flagtype}'])

                        # Name current Tab
                        this_sh.title = f'{short[adtype]} - {agtype} - {flagtype}'
                        
                        # Write Slice to the sheet and format
                        rows = dataframe_to_rows(working, index=False, header=True)
                        for r_idx, row in enumerate(rows, 1):
                            for c_idx, value in enumerate(row, 1):
                                this_sh.cell(row=r_idx+6, column=c_idx, value=value)

                        #Activate next sheet
                        this_sh = next_sh
                        next_sh = wb.copy_worksheet(this_sh)

        #Write Audit File
        if(exist):
            print('writing')
            #Remove extra copied sheets before writing
            for sn in wb.sheetnames:
                if 'Copy' in sn:
                    wb.remove(wb[sn])
            wb.save(filename=ssname)
            wb.close()

Starting Product: Active Detergent


/home/kneel/.local/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


writing
Starting Product: Dishwasher Cleaner
writing
Starting Product: Washing Machine Cleaner
writing
Starting Product: IP Sealing Ring 3 QT
writing
Starting Product: IP Sealing Ring 8 QT
writing
Starting Product: Silicone Bakeware Sling
writing
Starting Product: Steam Diverter DUO
writing
Starting Product: Exp Steamer Basket
writing
Starting Product: Steam Diverter LUX
writing
Starting Product: Cakeware Pans 4 7 9
writing
Starting Product: Cakeware Pans 4 Inch
writing
Starting Product: Cakeware Pans 7 Inch
writing
Starting Product: Cakeware Pans 9 Inch
writing
Starting Product: IP Sealing Ring 6 QT
writing
Starting Product: Silicone Roasting Rack
writing
Starting Product: IP Lids 6 QT
writing
Starting Product: IP Lids 8 QT
writing
Starting Product: Silicone Steamer Basket
writing
Starting Product: Foodi Lid Stand
writing
Starting Product: Silicone Egg Bite Molds
writing
Starting Product: Plant Stands
writing
Starting Product: Arch Plant Stand
writing
Starting Product: Floating Shelve